# End-to-End Multiclass Image Classification Example
1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
  1. [Permissions and environment variables](#Permissions-and-environment-variables)
  2. [Prepare the data](#Prepare-the-data)
3. [Training the model](#Training-the-model)
  1. [Training parameters](#Training-parameters)
  2. [Start the training](#Start-the-training)
4. [Compile](#Compile)
5. [Inference](#Inference)

## Introduction

이 데모에서는 Amazon Sagemaker 의 이미지 분류 알고리즘을 이용해서, [caltech-256 dataset](http://www.vision.caltech.edu/Image_Datasets/Caltech256/)를 학습합니다. 

시작하기에 앞서, 몇 가지 환경 설정과, 인증 설정이 필요합니다.

## Prequisites and Preprocessing

### Permissions and environment variables

AWS 서비스로의 연결과 권한 설정을 진행합니다. 이 단계의 세 가지 목표는:

* 학습과 데이터에 대한 역할 생성. 노트북 인스턴스를 생성할 때에, 자동으로 부여됩니다.
* 학습 데이터와 모델 데이터를 위한 S3 버킷
* Amazon SageMaker 이미지 분류 도커 이미지

In [ ]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "ic-fulltraining"

In [ ]:
from sagemaker import image_uris

training_image = image_uris.retrieve(
    region=sess.boto_region_name, framework="image-classification", version="latest"
)
print(training_image)

### Data preparation
학습 데이터를 다운받고, S3 에 업로드합니다. 이 데모에서는 256가지 종류로 나뉘어진 30607장의 이미지를 제공하는 [Caltech-256](http://www.vision.caltech.edu/Image_Datasets/Caltech256/) 데이터셋을 이용하여 학습을 진행합니다. 학습과 검증 데이터셋을 나누기 위해서, MXNet 예제와 같은 방식으로 데이터셋을 분할합니다. 각각의 클래스별로 60장의 이미지를 무작위로 선택하여 학습 데이터로 사용하고, 나머지 데이터는 검증 데이터로 이용합니다. Image Classification 알고리즘은 RecordIO 파일을 인풋으로 받습니다. 사용자는 이미지 파일을 인풋으로 제공할 수도 있으며, MXNet의 [im2rec](https://mxnet.incubator.apache.org/how_to/recordio.html?highlight=im2rec) 툴을 이용해서 RecordIO 형식으로 변환됩니다. p2.xlarge 인스턴스에서 전체 Caltech-256 데이터셋 (~1.2GB) 를 변환하는 데에 약 50초 정도가 소요됩니다. 하지만, 이 데모에서는 RecordIO 형식으로 제공되는 데이터셋을 이용합니다.

In [ ]:
import os
import urllib.request
import boto3


def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)


def upload_to_s3(channel, file):
    s3 = boto3.resource("s3")
    data = open(file, "rb")
    key = channel + "/" + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)


# caltech-256
download("http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec")
download("http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec")

In [ ]:
# Four channels: train, validation, train_lst, and validation_lst
s3train = "s3://{}/{}/train/".format(bucket, prefix)
s3validation = "s3://{}/{}/validation/".format(bucket, prefix)

# upload the lst files to train and validation channels
!aws s3 cp caltech-256-60-train.rec $s3train --quiet
!aws s3 cp caltech-256-60-val.rec $s3validation --quiet

학습 데이터가 적합한 형식으로 준비가 되면, 다음 단계는 데이터를 이용하여 학습을 진행하는 것 입니다. 학습 파라미터를 설정하고, 학습을 시작한 후, 진행 상황을 받아옵니다.

모든 설정을 마쳤으면, 사물 인식 모델을 학습할 준비가 되었습니다. 시작하기 위해, ``sageMaker.estimator.Estimator`` 객체를 생성합니다. Estimator 는 학습 작업을 시작합니다.

## 학습 파라미터
학습을 위해 설정되어야 할 파라미터에는 두 가지 종류가 있습니다. 첫 번째로는

* **학습 인스턴스 개수**: 학습을 위해 사용될 인스턴스의 개수입니다. 두 개 이상의 인스턴스를 사용할 경우, 분산 학습으로 진행됩니다.
* **학습 인스턴스 종류**: 학습을 위해 사용될 인스턴스의 종류입니다. 대부분 GPU 인스턴스를 사용합니다.
* **출력 경로**: 학습의 결과가 저장될 S3 폴더입니다.


In [ ]:
s3_output_location = "s3://{}/{}/output".format(bucket, prefix)
ic = sagemaker.estimator.Estimator(
    training_image,
    role,
    instance_count=1,
    instance_type="ml.p2.xlarge",
    volume_size=50,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    sagemaker_session=sess,
)


위의 파라미터와 별개로, 알고리즘마다 설정되는 하이퍼파라미터가 있습니다:

* **num_layers**: 뉴럴 네트워크의 레이어 개수(깊이). 이 데모에서는 18개의 레이어를 사용하지만, 50, 152 와 같이 큰 숫자도 사용 가능합니다.
* **image_shape**: 입력 이미지의 크기입니다. ‘채널 수, 높이, 너비’가 있으며, 실제 이미지 사이즈보다 커서는 안 되고, 채널 수는 실제 이미지의 채널 수와 같아야 합니다.
* **num_classes**: 데이터셋의 클래스 개수입니다. Imagenet 은 100개의 클래스로 학습되었지만, fine-tuning 과정에서 바뀔 수 있습니다. Caltech 데이터셋은 257개 (256개의 사물 카테고리 + 클러스터) 의 클래스를 사용합니다.
* **num_training_samples**: 학습 샘플의 개수입니다. Caltech 데이터셋은 분할 이후 15240개의 학습 샘플이 있습니다.
* **mini_batch_size**: 미니 배치에 있는 학습 샘플의 개수입니다. 분산 학습에서는 N * mini_batch_size 만큼의 학습 샘플이 배치마다 사용되고, 여기서 N은 학습에 사용된 인스턴스의 개수입니다.
* **epochs**: Epoch 의 수입니다.
* **learning_rate**: Learning rate 의 값입니다.
* **top_k**: 학습 중 top-k 정확도를 출력합니다.
* **precision_dtype**: 학습의 데이터형입니다. (Deafult: float32). ‘Float16’으로 설정될 경우, mixed_precision 모드로 학습이 진행되며 float32 모드보다 빠르게 학습이 될 수 있습니다.


In [ ]:
ic.set_hyperparameters(
    num_layers=18,
    image_shape="3,224,224",
    num_classes=257,
    num_training_samples=15420,
    mini_batch_size=128,
    epochs=5,
    learning_rate=0.01,
    top_k=2,
    precision_dtype="float32",
)

## Input data specification
데이터형과 채널을 설정합니다.

In [ ]:
train_data = sagemaker.inputs.TrainingInput(
    s3train,
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    s3validation,
    distribution="FullyReplicated",
    content_type="application/x-recordio",
    s3_data_type="S3Prefix",
)

data_channels = {"train": train_data, "validation": validation_data}

## Start the training
Fit 함수를 이용하여 학습을 시작합니다.

In [ ]:
ic.fit(inputs=data_channels, logs=True)

# Compile

***

[Amazon SageMaker Neo](https://aws.amazon.com/sagemaker/neo/) 은 모델이 두 배 빠르게 작동할 수 있도록 최적화를 시킵니다. `compile_model()`함수를 통해서, 인스턴스 종류 (m4) 와 컴파일된 모델이 저장될 s3 버킷을 설정합니다.

In [ ]:
output_path = "/".join(ic.output_path.split("/")[:-1])
optimized_ic = ic.compile_model(
    target_instance_family="ml_m4",
    input_shape={"data": [1, 3, 224, 224]},  # Batch size 1, 3 channels, 224x224 Images.
    output_path=output_path,
    framework="mxnet",
    framework_version="1.8",
    env={"MMS_DEFAULT_RESPONSE_TIMEOUT": "500"},
)
optimized_ic.image = image_uris.retrieve(
    region=sess.boto_region_name, framework="image-classification-neo", version="latest"
)
optimized_ic.name = "deployed-image-classification"

In [ ]:
from sagemaker.mxnet.model import MXNetModel

s3_custom_code_location = "s3://{}/{}/custom_code".format(bucket, prefix)

optimized_ic_model = MXNetModel(
    model_data=optimized_ic.model_data,
    image_uri=optimized_ic.image_uri,
    framework_version="1.8",
    role=role,
    sagemaker_session=sess,
    entry_point="inference.py",
    py_version="py37",
    env={"MMS_DEFAULT_RESPONSE_TIMEOUT": "500"},
    code_location=s3_custom_code_location,
)

# Inference

***

학습된 모델은 그 자체로서는 아무것도 하지 못합니다. 모델을 추론을 위해 사용할 수 있도록 배포합니다.

In [ ]:
ic_classifier = optimized_ic_model.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", use_compiled_model=True
)

### Download test image

In [ ]:
!wget -O test.jpg http://sagemaker-sample-files.s3.amazonaws.com/datasets/image/caltech-256/256_ObjectCategories/008.bathtub/008_0007.jpg
file_name = "test.jpg"
# test image
from IPython.display import Image

Image(file_name)

### Evaluation

추론을 위해서 이미지를 분석합니다. 모델은 이미지 분류 클래스에 대한 확률을 출력하고, 대부분 가장 큰 확률이 있는 클래스를 선택합니다.

**Note:** 현재는 모델의 성능이 좋지 않을 수 있습니다. 시간 절약을 위해서 이 데모에서는 5 epoch 동안만 학습을 진행하였고, 더 많은 epoch (20 정도) 동안 학습이 된다면, 모델의 성능이 향상될 것입니다.

In [ ]:
import json
import numpy as np
import PIL.Image

test_image = PIL.Image.open(file_name)
payload = np.asarray(test_image.resize((224, 224)))

result = ic_classifier.predict(payload)
# the result will output the probabilities for all classes
# find the class with maximum probability and print the class index
index = np.argmax(result)
object_categories = [
    "ak47",
    "american-flag",
    "backpack",
    "baseball-bat",
    "baseball-glove",
    "basketball-hoop",
    "bat",
    "bathtub",
    "bear",
    "beer-mug",
    "billiards",
    "binoculars",
    "birdbath",
    "blimp",
    "bonsai-101",
    "boom-box",
    "bowling-ball",
    "bowling-pin",
    "boxing-glove",
    "brain-101",
    "breadmaker",
    "buddha-101",
    "bulldozer",
    "butterfly",
    "cactus",
    "cake",
    "calculator",
    "camel",
    "cannon",
    "canoe",
    "car-tire",
    "cartman",
    "cd",
    "centipede",
    "cereal-box",
    "chandelier-101",
    "chess-board",
    "chimp",
    "chopsticks",
    "cockroach",
    "coffee-mug",
    "coffin",
    "coin",
    "comet",
    "computer-keyboard",
    "computer-monitor",
    "computer-mouse",
    "conch",
    "cormorant",
    "covered-wagon",
    "cowboy-hat",
    "crab-101",
    "desk-globe",
    "diamond-ring",
    "dice",
    "dog",
    "dolphin-101",
    "doorknob",
    "drinking-straw",
    "duck",
    "dumb-bell",
    "eiffel-tower",
    "electric-guitar-101",
    "elephant-101",
    "elk",
    "ewer-101",
    "eyeglasses",
    "fern",
    "fighter-jet",
    "fire-extinguisher",
    "fire-hydrant",
    "fire-truck",
    "fireworks",
    "flashlight",
    "floppy-disk",
    "football-helmet",
    "french-horn",
    "fried-egg",
    "frisbee",
    "frog",
    "frying-pan",
    "galaxy",
    "gas-pump",
    "giraffe",
    "goat",
    "golden-gate-bridge",
    "goldfish",
    "golf-ball",
    "goose",
    "gorilla",
    "grand-piano-101",
    "grapes",
    "grasshopper",
    "guitar-pick",
    "hamburger",
    "hammock",
    "harmonica",
    "harp",
    "harpsichord",
    "hawksbill-101",
    "head-phones",
    "helicopter-101",
    "hibiscus",
    "homer-simpson",
    "horse",
    "horseshoe-crab",
    "hot-air-balloon",
    "hot-dog",
    "hot-tub",
    "hourglass",
    "house-fly",
    "human-skeleton",
    "hummingbird",
    "ibis-101",
    "ice-cream-cone",
    "iguana",
    "ipod",
    "iris",
    "jesus-christ",
    "joy-stick",
    "kangaroo-101",
    "kayak",
    "ketch-101",
    "killer-whale",
    "knife",
    "ladder",
    "laptop-101",
    "lathe",
    "leopards-101",
    "license-plate",
    "lightbulb",
    "light-house",
    "lightning",
    "llama-101",
    "mailbox",
    "mandolin",
    "mars",
    "mattress",
    "megaphone",
    "menorah-101",
    "microscope",
    "microwave",
    "minaret",
    "minotaur",
    "motorbikes-101",
    "mountain-bike",
    "mushroom",
    "mussels",
    "necktie",
    "octopus",
    "ostrich",
    "owl",
    "palm-pilot",
    "palm-tree",
    "paperclip",
    "paper-shredder",
    "pci-card",
    "penguin",
    "people",
    "pez-dispenser",
    "photocopier",
    "picnic-table",
    "playing-card",
    "porcupine",
    "pram",
    "praying-mantis",
    "pyramid",
    "raccoon",
    "radio-telescope",
    "rainbow",
    "refrigerator",
    "revolver-101",
    "rifle",
    "rotary-phone",
    "roulette-wheel",
    "saddle",
    "saturn",
    "school-bus",
    "scorpion-101",
    "screwdriver",
    "segway",
    "self-propelled-lawn-mower",
    "sextant",
    "sheet-music",
    "skateboard",
    "skunk",
    "skyscraper",
    "smokestack",
    "snail",
    "snake",
    "sneaker",
    "snowmobile",
    "soccer-ball",
    "socks",
    "soda-can",
    "spaghetti",
    "speed-boat",
    "spider",
    "spoon",
    "stained-glass",
    "starfish-101",
    "steering-wheel",
    "stirrups",
    "sunflower-101",
    "superman",
    "sushi",
    "swan",
    "swiss-army-knife",
    "sword",
    "syringe",
    "tambourine",
    "teapot",
    "teddy-bear",
    "teepee",
    "telephone-box",
    "tennis-ball",
    "tennis-court",
    "tennis-racket",
    "theodolite",
    "toaster",
    "tomato",
    "tombstone",
    "top-hat",
    "touring-bike",
    "tower-pisa",
    "traffic-light",
    "treadmill",
    "triceratops",
    "tricycle",
    "trilobite-101",
    "tripod",
    "t-shirt",
    "tuning-fork",
    "tweezer",
    "umbrella-101",
    "unicorn",
    "vcr",
    "video-projector",
    "washing-machine",
    "watch-101",
    "waterfall",
    "watermelon",
    "welding-mask",
    "wheelbarrow",
    "windmill",
    "wine-bottle",
    "xylophone",
    "yarmulke",
    "yo-yo",
    "zebra",
    "airplanes-101",
    "car-side-101",
    "faces-easy-101",
    "greyhound",
    "tennis-shoes",
    "toad",
    "clutter",
]
print("Result: label - " + object_categories[index] + ", probability - " + str(result[index]))

### Clean up


데모 진행이 완료되었으면, 배포 엔드포인트를 삭제합니다. 아래 코드를 주석 해제한 후, 실행하여 엔드포인트와 모델을 삭제합니다.

In [ ]:
ic_classifier.delete_model()
ic_classifier.delete_endpoint()